In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pandas as pd
import io
import collections
spark = SparkSession.builder.master("local[2]").appName("dataframetest").getOrCreate()
df = spark.read.csv("sample_data/store_data.csv")
df.show()

+-----------------+------------+-------------+----------------+-------------+----------------+--------------+--------------+------------+--------------------+--------------+---------+-----+-----+-------------+------+-----------------+---------------+-------+---------+
|              _c0|         _c1|          _c2|             _c3|          _c4|             _c5|           _c6|           _c7|         _c8|                 _c9|          _c10|     _c11| _c12| _c13|         _c14|  _c15|             _c16|           _c17|   _c18|     _c19|
+-----------------+------------+-------------+----------------+-------------+----------------+--------------+--------------+------------+--------------------+--------------+---------+-----+-----+-------------+------+-----------------+---------------+-------+---------+
|           shrimp|     almonds|      avocado|  vegetables mix| green grapes|whole weat flour|          yams|cottage cheese|energy drink|        tomato juice|low fat yogurt|green tea|honey|sala

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)



In [ ]:
def generate_unique_item_set(df):
    # empty dataframe
    total_item_set_df = None
    
    # Iteration over each column - 20 columns
    for col_index in range(20):
        
        # Loading the elements of each column individually
        _c_df = df.select("_c" + str(col_index))
        
        if total_item_set_df is None:
            # None for the first iteration in the loop
            total_item_set_df = _c_df
            
        else:
            # After the first iteration, appending the entries from each column to total_item_set_df
            total_item_set_df = total_item_set_df.union(_c_df)
            
    
    return total_item_set_df.select("_c0").na.drop().rdd.flatMap(remove_duplicate_columns).distinct().toDF()

In [ ]:
# Function to remove the columns with duplicate values

def remove_duplicate_columns(x):
    # Length of the column
    col_len = len(x)
    
    # Empty RDD - set of values
    columns = set()
    
    # Removing any additional spaces from the elements and adding the elements into the column from RDD 'x'
    for col in range(col_len):
        x_col = str(x[col]).strip()
        columns.add(x_col)
    
    # To check if elements are present in the provided dataframe/RDD 
    if len(columns) < col_len:
        return []
    
    # Returning the sorted list of items in each element as tuple
    return [(tuple(sorted(columns)))]

In [ ]:
# Syntax structure: item_sets = function(dataframe)
item_sets = generate_unique_item_set(df)
item_sets.show()
# Give the number of unique items in the transaction dataset
print('item unique is:',item_sets.count())

+--------------------+
|                  _1|
+--------------------+
|   whole wheat pasta|
|           asparagus|
|            pancakes|
|         blueberries|
|            zucchini|
|              shrimp|
|             burgers|
|           spaghetti|
|         french wine|
|       strong cheese|
|extra dark chocolate|
|              melons|
|               cream|
|   frozen vegetables|
|           meatballs|
|          energy bar|
|            escalope|
|        energy drink|
|                mint|
|      vegetables mix|
+--------------------+
only showing top 20 rows

item unique is: 119


In [ ]:
# def get_freq_item_sets(total_records, candidate_sets_shared, transaction_df_rdd, min_support):    
#     #filtered_item_set = transaction_df_rdd.flatMap(lambda x: filter_and_map_transaction(x, candidate_sets_shared)) \
#     filtered_item_set = transaction_df_rdd.flatMap(lambda x: filter_and_map_transaction(x, candidate_sets_shared)).reduceByKey(lambda a, b: a+b).map(lambda x: (x[0], x[1]/total_records)).filter(lambda x: x[1]>=min_support)

#     return filtered_item_set

In [ ]:
# from pyspark.sql.types import StructType, ArrayType, StructField, DoubleType, StringType

# def apriori(item_sets, transaction_df_rdd, min_support):
    
#     """
#     Attributes
#     ----------
#     item_sets: DataFrame that has all the items present in the transactions
    
#     transaction_df_rdd: Transacations in the form of an RDD
    
#     min_support: Minimum support threshold
#     -----------
#     """
    
#     total_records = transaction_df_rdd.count()

#     freq_item_sets_all_orders = []

#     candidate_sets_order_1 = spark.sparkContext.broadcast(item_sets.collect())
    
#     frequent_item_sets_order_1 = get_freq_item_sets(total_records, candidate_sets_order_1, transaction_df_rdd, min_support)
#     freq_item_sets_all_orders.append(frequent_item_sets_order_1)
    
#     frequent_item_sets_order_1_df = frequent_item_sets_order_1.map(lambda x:tuple(x[0])).toDF()
    
#     k = 0
#     while is_freq_item_set_not_empty(freq_item_sets_all_orders[k]):
        
#         current_candidate_sets = get_all_possible_candidate_sets(freq_item_sets_all_orders[k], frequent_item_sets_order_1_df)
#         current_candidate_sets = spark.sparkContext.broadcast(current_candidate_sets.collect()) 
#         current_frequent_item_sets = get_freq_item_sets(total_records, current_candidate_sets, transaction_df_rdd, min_support)
        
#         freq_item_sets_all_orders.append(current_frequent_item_sets)
#         k += 1
#     return freq_item_sets_all_orders

In [ ]:
# Generate the frequent item set using the apriori function created above.
# Minimum support = 0.01
# Check the format in which each attribute must be provided

# transaction_df_rdd = df.rdd
# freq_item_sets_all_orders = apriori(item_sets, transaction_df_rdd, 0.01)

# print(freq_item_sets_all_orders)
# freq_item_sets_all_orders[0].take(20)

NameError: ignored